## 10_S2_TIF_collection
### Downloads the Sentinel 2 images for a given country adhering to the MGRS grid standard
### This notebook requires a lot of memory!
### Please see the last cell, which starts the image retrieval process, several configuration options can be defined there

### Initial configuration
#### To start working with this particular notebook, you need to provide necessary credential and settings
#### Below is an template of configuration, which is necessary prepare aside of this notebook and copy & paste all content in triple quotes to the next cell's input field
    """
    {
    "COS_ENDPOINT_URL": "s3.private.eu-de.cloud-object-storage.appdomain.cloud",
    "COS_AUTH_ENDPOINT_URL": "https://iam.cloud.ibm.com/oidc/token",
    "COS_APIKEY": "xxx",
    "UTILS_BUCKET": "notebook-utils-bucket",
    "COUNTRY_NAME": "Kenya",
    "GEOTIFF_BUCKET": "geotiffs"
    }
    """


In [1]:
# Read notebook configuration
import getpass
import json

config_str = getpass.getpass('Enter your prepared config: ')
config = json.loads(config_str)


In [2]:
# Import necessary libraries
import pandas as pd
import time
import ibm_boto3
from botocore.client import Config
from tqdm import tqdm
import threading
import os
import rasterio as rio
from io import BytesIO
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import geopandas as gpd
import pandas as pd
import json
import requests
from shapely import wkb, wkt
import shapely
import boto3
import random
import rioxarray
from shapely.prepared import prep
import numpy as np
import threading
from rasterio.mask import mask
from skimage import measure as M
from rasterio.plot import show
import glob
import mgrs
from datetime import datetime
import os
from botocore.config import Config
from botocore import UNSIGNED
from botocore.handlers import disable_signing

import traceback

m = mgrs.MGRS()

In [ ]:
#Make sure the boundary polygon exists for a given country
country = config["COUNTRY_NAME"]

In [ ]:
cos_client = ibm_boto3.client(service_name='s3',
                              ibm_api_key_id=config["COS_APIKEY"],
                              ibm_auth_endpoint=config["COS_AUTH_ENDPOINT_URL"],
                              config=Config(signature_version='oauth'),
                              endpoint_url=config["COS_ENDPOINT_URL"])

In [3]:
SENTINEL2_BUCKET_NAME = 'sentinel-cogs'
S3client = boto3.client('s3', region_name='us-west-2', config=Config(signature_version=UNSIGNED))

In [5]:
# get countries geojson
countries_geoJSON_url = 'https://datahub.io/core/geo-countries/r/countries.geojson'
countries_geoJSON = requests.get(countries_geoJSON_url).content
countries_geoJSON = json.loads(countries_geoJSON)

In [6]:
# extract Kenya geojson from counties
country_geometry = {}
for feature in countries_geoJSON['features']:
    if feature['properties']['ADMIN'] == country:
        country_geometry['geometry'] = feature['geometry']
        break


In [7]:
# read geometries to shapely format
if country_geometry['geometry']['type'] == 'MultiPolygon':
    coordinates = shapely.MultiPolygon(country_geometry['geometry']['coordinates'])
elif country_geometry['geometry']['type'] == 'Polygon':
    coordinates = shapely.Polygon(country_geometry['geometry']['coordinates'])
    
coordinates
list(coordinates.geoms)

[<POLYGON ((41.163 -2.091, 41.155 -2.112, 41.137 -2.115, 41.123 -2.094, 41.12...>,
 <POLYGON ((35.434 5.004, 35.396 4.926, 35.496 4.926, 35.571 4.904, 35.522 4....>]

In [8]:
def mask_off_clouds(TCI_tiff_filename, SCL_tiff_filename):
    '''
        This functions is aimed for removing cloud related segments from particulat tiff file
    '''
    
    try:
        with rio.open(SCL_tiff_filename) as src:
            
            data = src.read(1)
            
            # mark all unnecessary layers keep only cloud related
            try:
                data[data == 1] = 0
                data[data == 2] = 0
                data[data == 4] = 0
                data[data == 5] = 0
                data[data == 6] = 0
                data[data == 7] = 0

                data[data == 3] = 1
                data[data == 8] = 1
                data[data == 9] = 1
                data[data == 10] = 1

            except Exception as e:
                print(f"Class assign Exception ocurred for: {SCL_tiff_filename}")

            # retrieve contours of remained united layers
            contours = M.find_contours(data, 0.9)
            polygons = []
            
            # convert contours to lat lon representation
            for contour in contours:

                latlon_coords = []

                for coords in contour:
                    lon, lat = rio.transform.xy(src.transform, coords[0], coords[1])
                    latlon_coords.append([lon, lat])

                latlon_coords.append(latlon_coords[0])
                latlon_coords = shapely.Polygon(latlon_coords)
                
                polygons.append(latlon_coords)

            print(f"Cloud polygond were found: {len(polygons)}")

        # remove found cloud polygons from target colorful image
        with rio.open(TCI_tiff_filename, 'r') as TCI_src:
            
            if len(polygons) == 0:
                print("Zero polygons were found return nonmasked TCI")

                return TCI_src.read(), TCI_src.meta
            
            out_image, _ = rio.mask.mask(TCI_src, polygons, invert=True, all_touched=True, nodata=0)
            
            os.remove(TCI_tiff_filename)
            os.remove(SCL_tiff_filename)

            # image = np.array(out_image)
            # image[image == 0] = np.nan
            
            return out_image, TCI_src.meta
        
    except Exception as e:
        print(f"Exception occured: {e} for: {SCL_tiff_filename}")
        print(traceback.format_exc())
        
        return [], TCI_src.meta


In [9]:
# find all the MGRS tile names for given country polygon
MGRS_tiles = []

for polygon in coordinates.geoms:
    
    coordinates_list = polygon.exterior.coords._coords
    for xy in coordinates_list:
        MGRS_tiles.append(m.toMGRS(xy[1], xy[0], MGRSPrecision=0))

        
    latmin, lonmin, latmax, lonmax = polygon.bounds
    resolution = 0.05
    for lat in np.arange(latmin, latmax, resolution):
        for lon in np.arange(lonmin, lonmax, resolution):
            MGRS_tiles.append(m.toMGRS(round(lon,4), round(lat,4), MGRSPrecision=0))
        
MGRS_tiles = list(set(MGRS_tiles))

len(MGRS_tiles)

144

In [10]:
# reproject coordinate reference system from (x, y) pixels to (lat, lon) coordinates
def reproject_tif_CRS(filename: str):
    rds = rioxarray.open_rasterio(filename)
    rds_4326 = rds.rio.reproject("EPSG:4326")
    rds_4326.rio.to_raster(filename, compress="DEFLATE")

In [11]:
# download defined Sentinel2 TCI (True Color Image) and their according SCL (Scene Classification Layer) products from S3 bucket
def download_products(partial_tile_name):
    # bands = ['B02.tif', 'B03.tif', 'B04.tif']
    bands = ['TCI.tif']

    tiffs = []
    SCL_filename = None
    bucket_objects = [obj['Key'] for obj in S3client.list_objects(Bucket=SENTINEL2_BUCKET_NAME, Prefix=partial_tile_name)['Contents']]
                
    for obj_name in bucket_objects:
        if (str(obj_name[-7:]) in bands):
            tiffs.append(obj_name)

        elif (str(obj_name[-7:]) in ['SCL.tif']):
            SCL_filename = obj_name
    
    # download SCL layer and reproject its coordinate reference system
    filename = tiffs[0].split('/')[-2]
    SCL_file_path = f'tiles/{filename}_SCL.tif'
    S3client.download_file(SENTINEL2_BUCKET_NAME, SCL_filename, SCL_file_path)
    reproject_tif_CRS(SCL_file_path)  


    # download TCI layers and reproject its coordinate reference systemd 
    TCI_file_path = f'tiles/{filename}_TCI.tif'
    if 'TCI.tif' in bands:
        for tif in tiffs:
            S3client.download_file(SENTINEL2_BUCKET_NAME, tif, TCI_file_path)
            reproject_tif_CRS(TCI_file_path)
        
        return TCI_file_path, SCL_file_path
    

    # if bands were defined instead TCI download them combine and save
    tiles = []
    for tif in tqdm(tiffs, desc=f"Downloading product {tiffs[0].split('/')[-2]}", total=len(tiffs)):
        fileobj = S3client.get_object(Bucket=SENTINEL2_BUCKET_NAME, Key=tif)['Body']
        tile = rio.open(BytesIO(fileobj.read()), 'r')
        tiles.append(tile)

    # fileobj = S3client.get_object(Bucket=SENTINEL2_BUCKET_NAME, Key=SCL_filename)['Body']
    
    
    filename = tiffs[0].split('/')[-2]
    tif_file =f'{filename}.tif'
    tif_destination = f"tiles/{tif_file}"

    colorful_tile = rio.open(tif_destination, 'w', driver='Gtiff',
                            width=tiles[0].width, height=tiles[0].height,
                            count=3,
                            crs=tiles[0].crs,
                            transform=tiles[0].transform,
                            photometric="RGB", 
                            compress="DEFLATE", 
                            bigtiff="IF_NEEDED", 
                            tiled=True, 
                            blockxsize=256, 
                            blockysize=256,
                            dtype='uint8')

    for idx, tile in enumerate(tiles):
        colorful_tile.write(tile.read(1), idx + 1)
    
    colorful_tile.close()
    reproject_tif_CRS(tif_destination)

    return tif_destination, SCL_file_path


In [ ]:
# create folder for saved numpy arrays
masked_tiles_np_files_folder = 'masked_tiles_np_files'

if not os.path.exists(masked_tiles_np_files_folder):
    os.makedirs(masked_tiles_np_files_folder)


In [12]:
def process_product(product, masked_tiles_filenames, tiff_metadata):
    tci_tif_name, scl_tif_name = download_products(product)
    masked_tile, tiff_meta = mask_off_clouds(tci_tif_name, scl_tif_name)

    if len(masked_tile) == 0:
        pass
    else:
        try:
            filepath = f"{masked_tiles_np_files_folder}/{tci_tif_name.replace('.tif', '').replace('tiles/', '')}.npy"
            masked_tile = np.array(masked_tile).astype(np.float64)
            np.save(filepath, masked_tile)
            masked_tiles_filenames.append(filepath)
            tiff_metadata.append(tiff_meta)
        except Exception as e:
            print('Numpy saving error occurred', e)

In [ ]:
def download_tiles(
        MGRS_selected_tiles: list, 
        products_to_process: int,
        cloudy_percentage: int,
        years_range: list,
        months_range: list
        ):
    
    errors = []
    skippped_tiles = []

#     tiles = []

    for tile_idx, tile in enumerate(MGRS_selected_tiles):

        print(f"Processing tile: {tile} | {tile_idx+1} of {len(MGRS_selected_tiles)}")
        t1 = time.time()

        data_coverage_2b = {}
        cloud_coverage_percentages_2b = {}

        no_data_percentages_2a = {}
        cloud_coverage_percentages_2a = {}
        
        for year in years_range:
        
            for month in months_range:
                
                partial_tile_name = f"sentinel-s2-l2a-cogs/{tile[0:2]}/{tile[2:3]}/{tile[3:5]}/{year}/{month}"
                
                try:
            
                    bucket_objects = [obj['Key'] for obj in S3client.list_objects(Bucket=SENTINEL2_BUCKET_NAME, Prefix=partial_tile_name)['Contents']]
                    
                    for obj in bucket_objects:
                        if "L2A.json" in obj:
                            json_body = S3client.get_object(Bucket=SENTINEL2_BUCKET_NAME, Key=obj)['Body']
                            tile_info_metadata_json = json.loads(json_body.read().decode("utf-8"))
                            
                            key = '/'.join(obj.split('/')[:-1])
                            version = tile_info_metadata_json['stac_version']
                            if version == "1.0.0":
                                if "S2B_" in obj:
                                    try:
                                        created = datetime.strptime(tile_info_metadata_json["properties"]["created"], "%Y-%m-%dT%H:%M:%S.%f%z")
                                        no_data_percentage = float(tile_info_metadata_json["properties"]["s2:nodata_pixel_percentage"])
                                        tile_cloudy_percentage = float(tile_info_metadata_json["properties"]["eo:cloud_cover"])
                                        
                                    except Exception as e:
                                        print(f'S2B ver: {version} | Exception occured: {e} for product {obj}')
                                        pass

                                    if (no_data_percentage <= 2):
                                        data_coverage_2b[key] = no_data_percentage
                                        
                                    if (tile_cloudy_percentage <= cloudy_percentage):
                                        cloud_coverage_percentages_2b[key] = tile_cloudy_percentage

                                if "S2A_" in obj:

                                    try:
                                        created = datetime.strptime(tile_info_metadata_json["properties"]["created"], "%Y-%m-%dT%H:%M:%S.%f%z")
                                        no_data_percentage = tile_info_metadata_json["properties"]["s2:nodata_pixel_percentage"]
                                        tile_cloudy_percentage = tile_info_metadata_json["properties"]["eo:cloud_cover"]
                                        
                                    except Exception as e:
                                        print(f'S2A ver: {version} | Exception occured: {e} for product {obj}')
                                        pass
                                    
                                    if (no_data_percentage <= 2):
                                        no_data_percentages_2a[key] = no_data_percentage
                                            
                                    if (tile_cloudy_percentage <= cloudy_percentage):
                                        cloud_coverage_percentages_2a[key] = tile_cloudy_percentage
                            
                            elif version == "1.0.0-beta.2":

                                if "S2B_" in obj:
                                    try:

                                        if tile_info_metadata_json["properties"]['sentinel:valid_cloud_cover'] != False:
                                            created = datetime.strptime(tile_info_metadata_json["properties"]["datetime"], "%Y-%m-%dT%H:%M:%SZ")
                                            no_data_percentage = float(tile_info_metadata_json["properties"]["sentinel:data_coverage"])
                                            tile_cloudy_percentage = float(tile_info_metadata_json["properties"]["eo:cloud_cover"])
                                        
                                    except Exception as e:
                                        print(f'S2B ver: {version} | Exception occured: {e} for product {obj}')
                                        pass

                                    if (no_data_percentage >= 5):
                                        # print(version, key, created)
                                        data_coverage_2b[key] = 45 - no_data_percentage
                                        
                                    if (tile_cloudy_percentage <= cloudy_percentage):
                                        cloud_coverage_percentages_2b[key] = tile_cloudy_percentage

                                if "S2A_" in obj:

                                    try:
                                        if tile_info_metadata_json["properties"]['sentinel:valid_cloud_cover'] != False:
                                            created = datetime.strptime(tile_info_metadata_json["properties"]["datetime"], "%Y-%m-%dT%H:%M:%SZ")
                                            no_data_percentage = tile_info_metadata_json["properties"]["sentinel:data_coverage"]
                                            tile_cloudy_percentage = tile_info_metadata_json["properties"]["eo:cloud_cover"]
                                        
                                    except Exception as e:
                                        print(f'S2A ver: {version} | Exception occured: {e} for product {obj}')
                                        pass
                                    
                                    if (no_data_percentage >= 4):
                                        no_data_percentages_2a[key] = 45 - no_data_percentage
                                            
                                    if (tile_cloudy_percentage <= cloudy_percentage):
                                        cloud_coverage_percentages_2a[key] = tile_cloudy_percentage
                            
                except Exception as e:
                    
                    errors.append(partial_tile_name)
                    # print(f"Esception {e} occured /for: {partial_tile_name}")

        cloud_coverage_percentages_sorted_2b = sorted(cloud_coverage_percentages_2b.items(), key=lambda x: x[1])
        data_coverage_percentages_sorted_2b = sorted(data_coverage_2b.items(), key=lambda x:x[1])

        # data_coverage_percentages_sorted_2b.reverse()

        if len(data_coverage_percentages_sorted_2b) == 0:
            data_coverage_percentages_sorted_2b = cloud_coverage_percentages_sorted_2b[:products_to_process]
        if len(cloud_coverage_percentages_sorted_2b) == 0:
            cloud_coverage_percentages_sorted_2b = data_coverage_percentages_sorted_2b[:products_to_process]

        matched_products_2b = []

        for i in cloud_coverage_percentages_sorted_2b:
            if i[0] in [j[0] for j in data_coverage_percentages_sorted_2b]:
                matched_products_2b.append(i[0])

        print('matched_products_2b', len(matched_products_2b))

        matched_products_2b = matched_products_2b[:products_to_process]

        cloud_coverage_percentages_sorted_2a = sorted(cloud_coverage_percentages_2a.items(), key=lambda x: x[1])
        no_data_percentages_sorted_2a = sorted(no_data_percentages_2a.items(), key=lambda x:x[1])

        if len(no_data_percentages_sorted_2a) == 0:
            no_data_percentages_sorted_2a = cloud_coverage_percentages_sorted_2a[:products_to_process]
        if len(cloud_coverage_percentages_sorted_2a) == 0:
            cloud_coverage_percentages_sorted_2a = no_data_percentages_sorted_2a[:products_to_process]

        matched_products_2a = []

        for i in cloud_coverage_percentages_sorted_2a:
            if i[0] in [j[0] for j in no_data_percentages_sorted_2a]:
                matched_products_2a.append(i[0])
        
        print('matched_products_2a', len(matched_products_2a)) 

        matched_products_2a = matched_products_2a[:products_to_process]

        matched_products = matched_products_2a + matched_products_2b

        if len(matched_products) > 0:

            masked_tile_files = []

            tiff_metadata = []
            threads= []

            for idx, product in enumerate(matched_products, start=1):
                thread = threading.Thread(target=process_product, args=(product, masked_tile_files, tiff_metadata, ))
                threads.append(thread)

            # for idx, product in enumerate(matched_products_2b, start=1):
            #     thread = threading.Thread(target=process_product, args=(product, masked_tiles_2b, tiff_metadata, ))
            #     threads.append(thread)

            for thread in threads:
                thread.start()

            for tidx, thread in enumerate(threads, start=1):
                thread.join()
                
            print('masked_tiles', len(masked_tile_files))

            # memory optimized 
            
            # load first np array image product
            mean_of_tiffs = np.load(masked_tile_files[0])
            # replace 0 with np.nan values for nanmean func
            mean_of_tiffs[mean_of_tiffs == 0] = np.nan
            
            # rest arrays to process
            masked_tile_files_to_process = masked_tile_files[1:]
            
            for filepath in tqdm(masked_tile_files_to_process, desc='Calculating nanmean for tiles', total=len(masked_tile_files_to_process)):
                
                # load and remove .npy from memory
                tile_loaded = np.load(filepath)
                os.remove(filepath)
                # replace 0 with np.nan values for nanmean func
                tile_loaded[tile_loaded == 0] = np.nan
                
                # assemble ongoing array and processed array in np array
                tiles_to_process = np.array([mean_of_tiffs, tile_loaded])

                # clear from memory
                del tile_loaded
                
                # calculate mean of non nan values
                mean_of_tiffs = np.nanmean(tiles_to_process, axis=0)

            # convert as int16 type for raster image
            mean_of_tiffs = mean_of_tiffs.astype(np.int16)

            # save to raster .tif fi;e
            with rio.open(f'cloudless_tiffs/{tile}_cloudless.tif', 'w', **tiff_metadata[0]) as dest:
                dest.write(mean_of_tiffs)
            
            # upload to bucket
            try:
                res=cos_client.upload_file(Filename=f'cloudless_tiffs/{tile}_cloudless.tif', Bucket='s2lab2-temp',Key=f"{tile}_cloudless.tif")
                os.remove(f'cloudless_tiffs/{tile}_cloudless.tif')
                
            except Exception as e:
                print(Exception, e)
            else:
                print(f'{tile}_cloudless.tif Uploaded')

            del mean_of_tiffs
            
            print("Garbage collector: collected",
                      "%d objects." % gc.collect())
                
        print(f'TIFF tile processing time: {time.strftime("%H:%M:%S", time.gmtime(int(time.time() - t1)))}')

   

In [ ]:
# Manually define MGRS tiles to process, set this list before starting the computation
MGRS_selected = [
   # '44QMD',
   '44QLE',
   '43QHV',
   '43QHB',
   '43QCD'
]


# Run main function
download_tiles(
    MGRS_selected,
    products_to_process = 4, # amount of product to process 4 is optimal value to obtain clear image
    cloudy_percentage = 15,  #maximal cloud coverage percentage per product
    years_range = [2023, 2024], # years range can be from 2017 (its when Sentinel2B was launched)
    months_range = [7, 8, 9, 10, 11, 12] # desired months range from 1 to 12
    )
